In [1]:
import pandas as pd
import numpy as np
import pyreadstat as st
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm

In [2]:
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2024 3T\[MT06] Microeconometría I\Clases\Stata\Archivos Stata-20250107\laborsub.dta"

df, meta = st.read_dta(path)
df.head()

,lfp,whrs,kl6,k618,wa,we
0,1,1610,1,0,32,12
1,1,1656,0,2,30,12
2,1,1980,1,3,35,12
3,1,456,0,3,34,12
4,1,1568,1,2,31,14


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   lfp     250 non-null    int64
 1   whrs    250 non-null    int64
 2   kl6     250 non-null    int64
 3   k618    250 non-null    int64
 4   wa      250 non-null    int64
 5   we      250 non-null    int64
dtypes: int64(6)
memory usage: 11.8 KB


In [4]:
meta.column_labels

['1 if woman worked in 1975',
 "Wife's hours of work",
 '# of children younger than 6',
 '# of children between 6 and 18',
 "Wife's age",
 "Wife's educational attainment"]

In [5]:
df['lfp'].value_counts()

lfp
1    150
0    100
Name: count, dtype: int64

In [9]:
df[['whrs']].describe().T

,count,mean,std,min,25%,50%,75%,max
whrs,250.0,799.84,915.60348,0.0,0.0,406.5,1599.75,4950.0


In [10]:
#¿Qué sucede si queremos estimar una función de oferta de trabajo?
import statsmodels.api as sm

# Filtrar datos donde whrs > 0
df_filtered = df[df["whrs"] > 0]

# Definir variables dependiente (Y) e independientes (X)
y = df_filtered["whrs"]
X = df_filtered[["kl6", "k618", "wa", "we"]]

# Agregar constante para la intersección (equivalente a _cons en Stata)
X = sm.add_constant(X)

# Ajustar el modelo
model = sm.OLS(y, X).fit()

# Mostrar resultados
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   whrs   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     2.802
Date:                Thu, 20 Feb 2025   Prob (F-statistic):             0.0281
Time:                        23:26:34   Log-Likelihood:                -1214.6
No. Observations:                 150   AIC:                             2439.
Df Residuals:                     145   BIC:                             2454.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1629.8168    615.130      2.650      0.0

In [12]:
from statsmodels.discrete.count_model import Tobit

# Definir variables dependiente (Y) e independientes (X)
y = df["whrs"]
X = df[["kl6", "k618", "wa", "we"]]

# Agregar constante para la intersección (equivalente a _cons en Stata)
X = sm.add_constant(X)

# Ajustar el modelo Tobit con censura inferior en 0
model = Tobit(y, X).fit(censor_lower=0)

# Mostrar resultados
print(model.summary())

ImportError: cannot import name 'Tobit' from 'statsmodels.discrete.count_model' (c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\discrete\count_model.py)

In [16]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize

# Definir variables dependiente (Y) e independientes (X)
y = df["whrs"]
X = df[["kl6", "k618", "wa", "we"]]

# Agregar constante
X = sm.add_constant(X)

# Función de log-verosimilitud del modelo Tobit (censura inferior en 0)
def tobit_loglik(params, y, X):
    beta = params[:-1]  # Coeficientes de regresión
    sigma = params[-1]  # Desviación estándar del error
    
    if sigma <= 0:
        return np.inf  # Evitar valores no válidos
    
    xb = X @ beta  # Producto matricial X * beta
    uncensored = (y > 0)
    
    # Parte no censurada: Log-verosimilitud de la normal
    ll_uncensored = -0.5 * np.log(2 * np.pi) - np.log(sigma) - (y - xb) ** 2 / (2 * sigma**2)
    
    # Parte censurada: Probabilidad acumulada de la normal (usando scipy.stats.norm)
    ll_censored = np.log(np.maximum(norm.cdf(xb / sigma), 1e-10))  # Evita log(0)
    
    # Combinar log-verosimilitudes
    loglik = np.where(uncensored, ll_uncensored, ll_censored)
    
    return -np.sum(loglik)  # Maximizar la verosimilitud minimizando la función de pérdida

# Estimación de parámetros iniciales
init_params = np.append(np.zeros(X.shape[1]), 1)  # Coeficientes iniciales + sigma

# Optimización numérica
res = minimize(tobit_loglik, init_params, args=(y, X), method="BFGS")

# Resultados
beta_est = res.x[:-1]  # Coeficientes estimados
sigma_est = res.x[-1]  # Estimación de sigma

print("Coeficientes estimados:", beta_est)
print("Sigma estimado:", sigma_est)



c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_numdiff.py:592: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0


Coeficientes estimados: [-5200401.23618509  1717029.03177521   313985.12206049    94771.03642374
    44524.92114298]
Sigma estimado: 510055.98735519126


In [17]:
import math
import warnings

import numpy as np
import pandas as pd
from scipy.optimize import minimize
import scipy.stats
from scipy.special import log_ndtr
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error


def split_left_right_censored(x, y, cens):
    counts = cens.value_counts()
    if -1 not in counts and 1 not in counts:
        warnings.warn("No censored observations; use regression methods for uncensored data")
    xs = []
    ys = []

    for value in [-1, 0, 1]:
        if value in counts:
            split = cens == value
            y_split = np.squeeze(y[split].values)
            x_split = x[split].values

        else:
            y_split, x_split = None, None
        xs.append(x_split)
        ys.append(y_split)
    return xs, ys


def tobit_neg_log_likelihood(xs, ys, params):
    x_left, x_mid, x_right = xs
    y_left, y_mid, y_right = ys

    b = params[:-1]
    # s = math.exp(params[-1])
    s = params[-1]

    to_cat = []

    cens = False
    if y_left is not None:
        cens = True
        left = (y_left - np.dot(x_left, b))
        to_cat.append(left)
    if y_right is not None:
        cens = True
        right = (np.dot(x_right, b) - y_right)
        to_cat.append(right)
    if cens:
        concat_stats = np.concatenate(to_cat, axis=0) / s
        log_cum_norm = scipy.stats.norm.logcdf(concat_stats)  # log_ndtr(concat_stats)
        cens_sum = log_cum_norm.sum()
    else:
        cens_sum = 0

    if y_mid is not None:
        mid_stats = (y_mid - np.dot(x_mid, b)) / s
        mid = scipy.stats.norm.logpdf(mid_stats) - math.log(max(np.finfo('float').resolution, s))
        mid_sum = mid.sum()
    else:
        mid_sum = 0

    loglik = cens_sum + mid_sum

    return - loglik


def tobit_neg_log_likelihood_der(xs, ys, params):
    x_left, x_mid, x_right = xs
    y_left, y_mid, y_right = ys

    b = params[:-1]
    # s = math.exp(params[-1]) # in censReg, not using chain rule as below; they optimize in terms of log(s)
    s = params[-1]

    beta_jac = np.zeros(len(b))
    sigma_jac = 0

    if y_left is not None:
        left_stats = (y_left - np.dot(x_left, b)) / s
        l_pdf = scipy.stats.norm.logpdf(left_stats)
        l_cdf = log_ndtr(left_stats)
        left_frac = np.exp(l_pdf - l_cdf)
        beta_left = np.dot(left_frac, x_left / s)
        beta_jac -= beta_left

        left_sigma = np.dot(left_frac, left_stats)
        sigma_jac -= left_sigma

    if y_right is not None:
        right_stats = (np.dot(x_right, b) - y_right) / s
        r_pdf = scipy.stats.norm.logpdf(right_stats)
        r_cdf = log_ndtr(right_stats)
        right_frac = np.exp(r_pdf - r_cdf)
        beta_right = np.dot(right_frac, x_right / s)
        beta_jac += beta_right

        right_sigma = np.dot(right_frac, right_stats)
        sigma_jac -= right_sigma

    if y_mid is not None:
        mid_stats = (y_mid - np.dot(x_mid, b)) / s
        beta_mid = np.dot(mid_stats, x_mid / s)
        beta_jac += beta_mid

        mid_sigma = (np.square(mid_stats) - 1).sum()
        sigma_jac += mid_sigma

    combo_jac = np.append(beta_jac, sigma_jac / s)  # by chain rule, since the expression above is dloglik/dlogsigma

    return -combo_jac


class TobitModel:
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
        self.ols_coef_ = None
        self.ols_intercept = None
        self.coef_ = None
        self.intercept_ = None
        self.sigma_ = None

    def fit(self, x, y, cens, verbose=False):
        """
        Fit a maximum-likelihood Tobit regression
        :param x: Pandas DataFrame (n_samples, n_features): Data
        :param y: Pandas Series (n_samples,): Target
        :param cens: Pandas Series (n_samples,): -1 indicates left-censored samples, 0 for uncensored, 1 for right-censored
        :param verbose: boolean, show info from minimization
        :return:
        """
        x_copy = x.copy()
        if self.fit_intercept:
            x_copy.insert(0, 'intercept', 1.0)
        else:
            x_copy.scale(with_mean=True, with_std=False, copy=False)
        init_reg = LinearRegression(fit_intercept=False).fit(x_copy, y)
        b0 = init_reg.coef_
        y_pred = init_reg.predict(x_copy)
        resid = y - y_pred
        resid_var = np.var(resid)
        s0 = np.sqrt(resid_var)
        params0 = np.append(b0, s0)
        xs, ys = split_left_right_censored(x_copy, y, cens)

        result = minimize(lambda params: tobit_neg_log_likelihood(xs, ys, params), params0, method='BFGS',
                          jac=lambda params: tobit_neg_log_likelihood_der(xs, ys, params), options={'disp': verbose})
        if verbose:
            print(result)
        self.ols_coef_ = b0[1:]
        self.ols_intercept = b0[0]
        if self.fit_intercept:
            self.intercept_ = result.x[1]
            self.coef_ = result.x[1:-1]
        else:
            self.coef_ = result.x[:-1]
            self.intercept_ = 0
        self.sigma_ = result.x[-1]
        return self

    def predict(self, x):
        return self.intercept_ + np.dot(x, self.coef_)

    def score(self, x, y, scoring_function=mean_absolute_error):
        y_pred = np.dot(x, self.coef_)
        return scoring_function(y, y_pred)

In [20]:
y = df["whrs"]
X = df[["kl6", "k618", "wa", "we"]]
X = sm.add_constant(X)
df['cens'] = np.where(df['whrs'] == 0, -1, 0)
cens = df['cens']
tr = TobitModel()
tr = tr.fit(X, y, cens, verbose=False)
tr.coef_

array([ 294.44827939, -827.78090417, -140.01430074,  -24.97854148,
        103.69518044])